In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the gene expression data and transpose it
gene_expression = pd.read_csv("normalized_counts_vst.csv").T
gene_expression.columns = gene_expression.iloc[0].tolist()
gene_expression = gene_expression.iloc[1:]
gene_expression['Sample_ID'] = gene_expression.index

# Load the metadata with the target variable
metadata = pd.read_csv("metadata.csv")

# Merge the gene expression data with the metadata based on 'Sample_ID'
merged_data = pd.merge(gene_expression, metadata, on='Sample_ID')

# Convert the 'Host' column to binary encoding (1 for 'MCAV', 0 otherwise)
merged_data['Host'] = merged_data['Host'].apply(lambda x: 1 if x == 'OANN' else 0)

# Convert columns to numeric type where possible
for col in merged_data.columns[:-1]:
    merged_data[col] = pd.to_numeric(merged_data[col], errors='coerce')

# Calculate correlation between features and the 'Host'
ccorrelations = merged_data.corr()['Host'].sort_values(ascending=False)

# Select genes/features that are positively correlated with 'Host'
selected_genes = correlations.drop('Host').index[correlations.drop('Host') > 0].tolist()

# Prepare X and y for model training
X = merged_data[selected_genes]
y = merged_data['Host']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


/var/folders/ln/f99l0sv10qngngkhp5fwp5cw0000gn/T/ipykernel_21085/3853315778.py:24: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  ccorrelations = merged_data.corr()['Host'].sort_values(ascending=False)


In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Create a Logistic Regression model
logistic_model = LogisticRegression()

# Fit the model
logistic_model.fit(X_train, y_train)

# Predict on test set
logistic_predictions = logistic_model.predict(X_test)

# Calculate accuracy
logistic_accuracy = accuracy_score(y_test, logistic_predictions)
print("Logistic Regression Accuracy:", logistic_accuracy)

# Identify two genes significantly correlated with 'Host'
top_genes_logistic = correlations.drop('Host').nlargest(2).index.tolist()
print("Top correlated genes for Logistic Regression:", top_genes_logistic)
# Get absolute coefficients and count significant ones
threshold = 0.1  # Set the threshold for significant coefficients
significant_genes_count = sum(abs(logistic_model.coef_[0]) > threshold)

# Print coefficients for further inspection
print("Coefficients from Logistic Regression:")
print(logistic_model.coef_)

print("Number of significant genes with Logistic Regression:", significant_genes_count)

Logistic Regression Accuracy: 0.8888888888888888
Top correlated genes for Logistic Regression: ['Q8C196', 'Q6PAZ3']
Coefficients from Logistic Regression:
[[-0.01334107 -0.00570983 -0.00525592 ...  0.00435485 -0.02062964
  -0.00692578]]
Number of significant genes with Logistic Regression: 0


/Users/jahnavi/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [26]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda_model = LinearDiscriminantAnalysis()
lda_model.fit(X_train, y_train)
# Fit the LDA model
lda_model.fit(X_train, y_train)

# Get the number of significant components
significant_components_count = sum(lda_model.explained_variance_ratio_ > threshold)

# Predict on the test set
lda_predictions = lda_model.predict(X_test)

# Calculate accuracy
lda_accuracy = accuracy_score(y_test, lda_predictions)
print("LDA:", logistic_accuracy)

significant_components_count = sum(lda_model.explained_variance_ratio_ > threshold)
print("Number of significant components with LDA:", significant_components_count)


LDA: 0.8888888888888888
Number of significant components with LDA: 1
